# PyCity Schools Analysis
  - In the entire district, the overall passing percentage for the combined math and reading scores is only 65.2%.
  - Cabrera High School was the school the highest overall passing rate at 91.3%.
  - Rodriguez High School was the school the lowest overall passing rate at 53.0%.
  - The top 5 performing schools were all charter schools, while the bottom 5 performing schools were all district schools.

In [1]:
#Dependencies
import pandas as pd

In [2]:
#File Paths
file_1 = "Resources/schools_complete.csv"
file_2 = "Resources/students_complete.csv"

#File Reads
school_file = pd.DataFrame(pd.read_csv(file_1))
student_file = pd.DataFrame(pd.read_csv(file_2))

#Rename columns for merge 
school_file.rename(columns={"name":"school"}, inplace=True)

#Merge files together into one DataFrame
merged_file = pd.merge(school_file, student_file, left_index=True, on="school")
merged_file.head()

,School ID,school,type,size,budget,Student ID,name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


### District Summary

In [3]:
#------------------District Summary----------------
#Calculate "Total Schools"
total_schools = len(merged_file["school"].unique())
#Calculate "Total Students"
total_students = merged_file["size"].unique().sum()
#Calculate "Total Budget"
total_budget = merged_file["budget"].unique().sum()
#Calculate "Average Math Score"
aver_math_score = merged_file["math_score"].mean()
#Calculate "Average Reading Score"
aver_reading_score = merged_file["reading_score"].mean()
#Calculate "% Passing Math"
amount_passing_m = merged_file.loc[merged_file["math_score"] >= 70]["math_score"].count()
percent_passing_math = amount_passing_m/total_students
#Calculate "% Passing Reading"
amount_passing_r = merged_file.loc[merged_file["reading_score"] >= 70]["reading_score"].count()
percent_passing_reading = amount_passing_r/total_students
#Calculate "Overall Passing Rate"
amount_passing_both = merged_file[(merged_file['math_score'] >= 70) & (merged_file['reading_score'] >= 70)]["name"].count()
overall_passing_rate = amount_passing_both/total_students

#Create DataFrame for "District Summary" 
district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [aver_math_score],
    "Average Reading Score": [aver_reading_score],
    "% Passing Math":[percent_passing_math],
    "% Passing Reading": [percent_passing_reading],
    "Overall Passing Rate": [overall_passing_rate]}, columns=["Total Schools","Total Students",
                                                              "Total Budget","Average Math Score",
                                                              "Average Reading Score","% Passing Math",
                                                              "% Passing Reading","Overall Passing Rate"])
#Format "District Summary DataFrame"
district_summary_df.style.format({"Total Students":"{:,}",
                                  "Total Budget":"${:,.2f}",
                                  "Average Math Score":"{:.1f}",
                                  "Average Reading Score":"{:.1f}",
                                  "% Passing Math":"{:.1%}",
                                  "% Passing Reading":"{:.1%}",
                                  "Overall Passing Rate":"{:.1%}"})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",79.0,81.9,75.0%,85.8%,65.2%


### School Summary

In [4]:
#Group data by school and set as index
school = merged_file.set_index("school").groupby(["school"])
#Calculate "School Type"
school_types = school_file.set_index("school")["type"]
#Calculate "Total Students" per school
total_students_per_school = school["Student ID"].count()
#Calculate "Total School Budget" per school
school_budget = school_file.set_index("school")["budget"]
#Calculate "Per Student Budget" per school
student_budget = school_file.set_index("school")["budget"] / school_file.set_index("school")["size"]
#Calculate "Average Math Score" per school
math_score_school = school["math_score"].mean()
#Calculate "Average Reading Score" per school
reading_score_school = school["reading_score"].mean()
#Calculate "% Passing Math" per school
passing_math = merged_file[merged_file["math_score"] >= 70].groupby("school")["Student ID"].count() / total_students_per_school
#Calculate "% Passing reading" per school
passing_reading = merged_file[merged_file["reading_score"] >= 70].groupby("school")["Student ID"].count() / total_students_per_school
#Calculate "% Overall Passing Rate" per school
overall_passing_school = merged_file[(merged_file["math_score"] >= 70) & (merged_file["reading_score"] >= 70)].groupby("school")["Student ID"].count() / total_students_per_school 

#Create DataFrame for "School Summary"
school_summary = pd.DataFrame({
    "School Type":school_types,
    "Total Students":total_students_per_school,
    "Total School Budget":school_budget, 
    "Per Student Budget": student_budget,
    "Average Math Score": math_score_school,
    "Average Reading Score": reading_score_school,
    "% Passing Math": passing_math,
    "% Passing reading": passing_reading,
    "% Overall Passing Rate": overall_passing_school}, columns=["School Type",
                                                                "Total Students",
                                                                "Total School Budget",
                                                                "Per Student Budget",
                                                                "Average Math Score",
                                                                "Average Reading Score",
                                                                "% Passing Math",
                                                                "% Passing reading",
                                                                "% Overall Passing Rate"])
                                                                                           

    
#Format "School Summary" DataFrame                                                                                            
school_summary.style.format({"Total Students": "{:,}",
                             "Total School Budget": "${:,}", 
                             "Per Student Budget": "${:.0f}",
                             "Average Math Score": "{:.1f}",
                             "Average Reading Score": "{:.1f}",
                             "% Passing Math": "{:.1%}",
                             "% Passing reading": "{:.1%}",
                             "% Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928",$628,77.0,81.0,66.7%,81.9%,54.6%
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,91.3%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,53.2%
Ford High School,District,"2,739","$1,763,916",$644,77.1,80.7,68.3%,79.3%,54.3%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,90.6%
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,53.5%
Holden High School,Charter,427,"$248,087",$581,83.8,83.8,92.5%,96.3%,89.2%
Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,66.1%,81.2%,53.5%
Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,90.5%


### Top Performing Schools (By Overall Passing Rate)

In [5]:
#Sort by "% Overall Passing Rate"
top5_schools = school_summary.sort_values("% Overall Passing Rate", ascending=False)
top5_schools.head()

#Display top 5 performing school
top5_schools = top5_schools.head()

#Format "Top Performing Schools (By Overall Passing Rate)" DataFrame                                                                                            
top5_schools.style.format({"Total Students": "{:,}",
                             "Total School Budget": "${:,}", 
                             "Per Student Budget": "${:.0f}",
                             "Average Math Score": "{:.1f}",
                             "Average Reading Score": "{:.1f}",
                             "% Passing Math": "{:.1%}",
                             "% Passing reading": "{:.1%}",
                             "% Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,91.3%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4,83.8,93.3%,97.3%,90.9%
Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,84.0,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,90.5%


### Bottom Performing Schools (By Overall Passing Rate)

In [6]:
#Sort by "% Overall Passing Rate"
bottom5_schools = school_summary.sort_values("% Overall Passing Rate", ascending=True)
bottom5_schools.head()

#Display bottom 5 performing school
bottom5_schools = bottom5_schools.head()

#Format "Lowest Performing Schools (By Overall Passing Rate)" DataFrame                                                                                            
bottom5_schools.style.format({"Total Students": "{:,}",
                             "Total School Budget": "${:,}", 
                             "Per Student Budget": "${:.0f}",
                             "Average Math Score": "{:.1f}",
                             "Average Reading Score": "{:.1f}",
                             "% Passing Math": "{:.1%}",
                             "% Passing reading": "{:.1%}",
                             "% Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8,80.7,66.4%,80.2%,53.0%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,53.2%
Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,53.5%
Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,53.5%
Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,66.1%,81.2%,53.5%


### Math Scores By Grade

In [7]:
#Calculate average math scores for each grade level
ninth_math = student_file.loc[student_file["grade"] == "9th"].groupby("school")["math_score"].mean()
tenth_math = student_file.loc[student_file["grade"] == "10th"].groupby("school")["math_score"].mean()
eleventh_math = student_file.loc[student_file["grade"] == "11th"].groupby("school")["math_score"].mean()
twelfth_math = student_file.loc[student_file["grade"] == "12th"].groupby("school")["math_score"].mean()

#Create "Math Scores By Grade" DataFrame
math_scores = pd.DataFrame({"9th": ninth_math,
                            "10th": tenth_math,
                            "11th": eleventh_math,
                            "12th": twelfth_math}, columns=["9th", "10th", "11th", "12th"])

#Hide index title for aesthetics 
math_scores.index.name = " "

#Format "Math Scores By Grade" DataFrame
math_scores.style.format({"9th": "{:.1f}",
                           "10th": "{:.1f}",
                           "11th": "{:.1f}",
                           "12th": "{:.1f}"})


,9th,10th,11th,12th
,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


### Reading Scores By Grade

In [8]:
#Calculate average reading scores for each grade level
ninth_reading = student_file.loc[student_file["grade"] == "9th"].groupby("school")["reading_score"].mean()
tenth_reading = student_file.loc[student_file["grade"] == "10th"].groupby("school")["reading_score"].mean()
eleventh_reading = student_file.loc[student_file["grade"] == "11th"].groupby("school")["reading_score"].mean()
twelfth_reading = student_file.loc[student_file["grade"] == "12th"].groupby("school")["reading_score"].mean()

#Create "Reading Scores By Grade" DataFrame
math_scores = pd.DataFrame({"9th": ninth_reading,
                            "10th": tenth_reading,
                            "11th": eleventh_reading,
                            "12th": twelfth_reading}, columns=["9th", "10th", "11th", "12th"])

#Hide index title for aesthetics 
math_scores.index.name = " "

#Format "Reading Scores By Grade" DataFrame
math_scores.style.format({"9th": "{:.1f}",
                           "10th": "{:.1f}",
                           "11th": "{:.1f}",
                           "12th": "{:.1f}"})

,9th,10th,11th,12th
,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


### Scores By School Spending

In [9]:
#Create bins for data to be held in
bins = [0,585,615,645,675]
#Create titles for each bin
bin_names = ["<$585","$585-615","$615-645","$645-675"]
#Cut the calculation for spending per student to merged file under new column "spending_bins"
merged_file["spending_bins"] = pd.cut(merged_file["budget"]/merged_file["size"], bins, labels = bin_names)

#Group by spending bins
spending = merged_file.groupby("spending_bins")

#Calculate "Average Math Score"
av_math_score_per_bin = spending["math_score"].mean()
#Calculate "Average Reading Score"
av_reading_score_per_bin = spending["reading_score"].mean()
#Calculate "% Passing Math"
pass_math_per_bin = merged_file[merged_file["math_score"] >= 70].groupby("spending_bins")["Student ID"].count() / spending["Student ID"].count()
#Calculate "% Passing Reading"
pass_reading_per_bin = merged_file[merged_file["reading_score"] >= 70].groupby("spending_bins")["Student ID"].count() / spending["Student ID"].count()
#Calculate "% Overall Passing Rate"
pass_overall_per_bin = merged_file[(merged_file["math_score"] >= 70) & (merged_file["reading_score"] >= 70)].groupby("spending_bins")["Student ID"].count() / spending["Student ID"].count()

#Create "Scores By School Spending" DataFrame
scores_by_spending = pd.DataFrame({"Average Math Score": av_math_score_per_bin,
                                   "Average Reading Score": av_reading_score_per_bin,
                                   "% Passing Math": pass_math_per_bin,
                                   "% Passing Reading": pass_reading_per_bin, 
                                   "% Overall Passing Rate": pass_overall_per_bin}, columns=["Average Math Score",
                                                                                             "Average Reading Score",
                                                                                             "% Passing Math",
                                                                                             "% Passing Reading", 
                                                                                             "% Overall Passing Rate"])
#Hide index title for aesthetics 
scores_by_spending.index.name = " "

#Format "Scores By School Spending" DataFrame
scores_by_spending.style.format({"Average Math Score": "{:.1f}", 
                                 "Average Reading Score": "{:.1f}", 
                                 "% Passing Math": "{:.1%}", 
                                 "% Passing Reading": "{:.1%}",
                                 "% Overall Passing Rate": "{:.1%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,
<$585,83.4,84.0,93.7%,96.7%,90.6%
$585-615,83.5,83.8,94.1%,95.9%,90.1%
$615-645,78.1,81.4,71.4%,83.6%,60.3%
$645-675,77.0,81.0,66.2%,81.1%,53.5%


### Scores By School Size

In [10]:
#Create bins for data to be held in
bins = [0,1000,2000,5000]
#Create titles for each bin
bin_names = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]
#Cut size data to merged file under new column "size_bins"
merged_file["size_bins"] = pd.cut(merged_file["size"], bins, labels = bin_names)

#Group by size bins
size = merged_file.groupby("size_bins")

#Calculate "Average Math Score"
av_math_score_per_bin = size["math_score"].mean()
#Calculate "Average Reading Score"
av_reading_score_per_bin = size["reading_score"].mean()
#Calculate "% Passing Math"
pass_math_per_bin = merged_file[merged_file["math_score"] >= 70].groupby("size_bins")["Student ID"].count() / size["Student ID"].count()
#Calculate "% Passing Reading"
pass_reading_per_bin = merged_file[merged_file["reading_score"] >= 70].groupby("size_bins")["Student ID"].count() / size["Student ID"].count()
#Calculate "% Overall Passing Rate"
pass_overall_per_bin = merged_file[(merged_file["math_score"] >= 70) & (merged_file["reading_score"] >= 70)].groupby("size_bins")["Student ID"].count() / size["Student ID"].count()

#Create "Scores By School Size" DataFrame
scores_by_size = pd.DataFrame({"Average Math Score": av_math_score_per_bin,
                               "Average Reading Score": av_reading_score_per_bin,
                               "% Passing Math": pass_math_per_bin,
                               "% Passing Reading": pass_reading_per_bin, 
                               "% Overall Passing Rate": pass_overall_per_bin}, columns=["Average Math Score",
                                                                                         "Average Reading Score",
                                                                                         "% Passing Math",
                                                                                         "% Passing Reading", 
                                                                                         "% Overall Passing Rate"])
#Hide index title for aesthetics 
scores_by_size.index.name = " "

#Format "Scores By School Size" DataFrame
scores_by_size.style.format({"Average Math Score": "{:.1f}", 
                             "Average Reading Score": "{:.1f}", 
                             "% Passing Math": "{:.1%}", 
                             "% Passing Reading": "{:.1%}",
                             "% Overall Passing Rate": "{:.1%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,
Small (<1000),83.8,84.0,94.0%,96.0%,90.1%
Medium (1000-2000),83.4,83.9,93.6%,96.8%,90.6%
Large (2000-5000),77.5,81.2,68.7%,82.1%,56.6%


### Scores By School Type

In [11]:
#Group by school type
by_type = merged_file.groupby("type")

#Calculate "Average Math Score"
av_math_score_per_type = by_type["math_score"].mean()
#Calculate "Average Reading Score"
av_reading_score_per_type = by_type["reading_score"].mean()
#Calculate "% Passing Math"
pass_math_per_type = merged_file[merged_file["math_score"] >= 70].groupby("type")["Student ID"].count() / by_type["Student ID"].count()
#Calculate "% Passing Reading"
pass_reading_per_type = merged_file[merged_file["reading_score"] >= 70].groupby("type")["Student ID"].count() / by_type["Student ID"].count()
#Calculate "% Overall Passing Rate"
pass_overall_per_type = merged_file[(merged_file["math_score"] >= 70) & (merged_file["reading_score"] >= 70)].groupby("type")["Student ID"].count() / by_type["Student ID"].count()

#Create "Scores By School Type" DataFrame
scores_by_type = pd.DataFrame({"Average Math Score": av_math_score_per_type,
                               "Average Reading Score": av_reading_score_per_type,
                               "% Passing Math": pass_math_per_type,
                               "% Passing Reading": pass_reading_per_type, 
                               "% Overall Passing Rate": pass_overall_per_type}, columns=["Average Math Score",
                                                                                         "Average Reading Score",
                                                                                         "% Passing Math",
                                                                                         "% Passing Reading", 
                                                                                         "% Overall Passing Rate"])
#Hide index title for aesthetics 
scores_by_type.index.name = " "

#Format "Scores By School Type" DataFrame
scores_by_type.style.format({"Average Math Score": "{:.1f}", 
                             "Average Reading Score": "{:.1f}", 
                             "% Passing Math": "{:.1%}", 
                             "% Passing Reading": "{:.1%}",
                             "% Overall Passing Rate": "{:.1%}"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,
Charter,83.4,83.9,93.7%,96.6%,90.6%
District,77.0,81.0,66.5%,80.9%,53.7%
